In [2]:
import numpy as np
import scipy.stats as spst
import scipy.optimize as spop
import scipy.interpolate as spint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model
from scipy import stats
import textwrap
from scipy.interpolate import interp1d


#  Pefil CTD1 (XBT)

In [47]:
est = input('Nome da Estacao:')
num = input('Numero da Estacao:')
loc = r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\Analises\CTDxCTD\perfil_'+num+r'\xbt_'+num+'.xlsx'
df = pd.read_excel(loc)
df.head(5)

Nome da Estacao:MCT030
Numero da Estacao:30


,prof_xbt,temp_xbt
0,5,31.8038
1,6,31.8132
2,7,31.8822
3,8,32.0057
4,9,32.0561


In [3]:
plt.figure(figsize=(6, 8))
plt.plot(df['temp_xbt'],df['prof_xbt'], label='Dados observados')
plt.title('Perfil XBT Estação'+' '+est, fontsize = 20)
plt.xlabel('Temperatura[°C]', fontsize = 15)
plt.ylabel('Profundidade', fontsize = 15)
plt.ylim(600, 0)
plt.legend(loc='lower left', shadow=True, fontsize='large')
plt.show()
#plt.savefig(r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\regressaolinear\ANALISE\perfil_'+num+r'\xbt_'+num+'.png')

# Perfil CTD2 (CTDMB) 

In [48]:
local = r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\Analises\CTDxCTD\perfil_'+num+r'\ctd_'+num+'.xlsx'
df2 = pd.read_excel(local)
df2.head(5)

,prof_ctd,temp_ctd
0,3,31.8487
1,4,31.8505
2,5,31.8579
3,6,31.8770
4,7,31.9020


In [5]:
plt.figure(figsize= (6,8))
plt.plot(df2['temp_ctd'], df2['prof_ctd'], 'g',label='Dados observados')
plt.title('Perfil CTD Estação'+' '+est, fontsize = 20)
plt.xlabel('Temperatura [ITS-90, deg C]', fontsize = 15)
plt.ylabel('Profundidade [salt water, m]', fontsize = 15)
plt.ylim(600, 0)
plt.legend(loc='lower left', shadow=True, fontsize='large')
plt.show()

# CTD x XBT

In [49]:
plt.figure(figsize=(6, 8))
plt.plot(df['temp_xbt'],df['prof_xbt'], 'b', label="Perfil XBT", linewidth=5)
plt.plot(df2['temp_ctd'], df2['prof_ctd'], 'r', label="Perfil CTD", linewidth=5)
plt.ylim(600, 0)
plt.xlim(30, 40)
plt.title('Perfil XBT x CTD Estação'+' '+est, fontsize = 20)
plt.xlabel('Temperatura[°C]', fontsize = 15)
plt.ylabel('Profundidade[m]', fontsize = 15)
plt.legend(loc='lower left', shadow=True, fontsize='large')
plt.show()

# Interpolacao

Primeiro tratar os dados para o comecar nas mesmas profundidades e terminar nas respectivas. (Evitar os NaN s) 


In [50]:
localint = r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\Analises\CTDxCTD\perfil_'+num+r'\xbt_ctd_'+num+'.xlsx'
dfint = pd.read_excel(localint)
dfint.head(4)

,prof_xbt,temp_xbt,prof_ctd,temp_ctd
0,5,31.8038,5,31.8579
1,6,31.8132,6,31.8770
2,7,31.8822,7,31.9020
3,8,32.0057,8,31.9162


In [51]:
dff = []
for i in dfint['temp_ctd']:
    dff.append(i)
dff2 = []
for i in dfint['prof_ctd']:
    dff2.append(i)
dfint3 = pd.DataFrame(data = dff2, columns = ['prof_ctd'])
dfint3['temp_ctd'] = dff

dfint33 = dfint3.dropna()


x = dfint['prof_xbt']
y = dfint['temp_xbt']
yi = dfint33['prof_ctd']
xi = np.interp(yi, x, y)
plt.figure()
plt.plot(dfint33['temp_ctd'] , -dfint33['prof_ctd'] , '-ob', label = ' CTD')
plt.plot(xi, -yi, '-or', label = 'XBT interpolado')
plt.plot(dfint['temp_xbt'] , -dfint['prof_xbt'], '-og', label = ' XBT')
plt.legend(loc='upper left', shadow=True, fontsize='large')
#plt.ylim(600, 0)
plt.show()

In [52]:
a = len(xi)
b = len(x)
c = len(yi)
d = len(y)
if a < b:
    print('CORRETO, CONTINUE!!!')
print('xi =',a,'; x =',b,'; yi =', c,'; y =',d)

xi = 58 ; x = 58 ; yi = 58 ; y = 58


In [53]:
dfinterpolado = pd.DataFrame(data = xi, columns = ['xbt_inter'])
dfinterpolado.head(5)

,xbt_inter
0,31.8038
1,31.8132
2,31.8822
3,32.0057
4,32.0561


# Regressao Linear

ctd = dfint33 e xbt interpolado = dfinterpolado

In [54]:
dfinterpolado['ctd'] = dfint33['temp_ctd']
dfinterpolado.head(5)


,xbt_inter,ctd
0,31.8038,31.8579
1,31.8132,31.8770
2,31.8822,31.9020
3,32.0057,31.9162
4,32.0561,31.9511


In [25]:
dfinterpolado.describe()

,xbt_inter,ctd
count,68.000000,68.000000
mean,35.161596,35.058216
std,1.106182,1.134837
min,32.536300,32.551500
25%,34.718925,34.648900
50%,35.463750,35.286050
75%,36.003600,35.942850
max,36.359500,36.271100


In [55]:
minixbt = dfinterpolado['xbt_inter'].min()
minictd = dfinterpolado['ctd'].min()
maxixbt = dfinterpolado['xbt_inter'].max()
maxictd = dfinterpolado['ctd'].max()

npMatrix = np.matrix(dfinterpolado)
X, Y = npMatrix[:,1], npMatrix[:,0]
mdl = LinearRegression().fit(X,Y) # either this or the next line
#mdl = LinearRegression().fit(filtered_data[['x']],filtered_data.y)
m = mdl.coef_[0]
b = mdl.intercept_
#print("formula: y = {0}x + {1}".format(m, b)) # following slope intercept form 
stm = str(m)
stb = str(b)
form = ("y = {0}x + {1}".format(stm[1:len(stm)-1], stb[1:len(stb)-1]))
print(form)

y =  0.85579431x +  4.73141176


In [56]:
##Estatistica, regressao linear coeficientes
x = dfinterpolado["ctd"]
y = dfinterpolado["xbt_inter"]
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
a = slope
bb = intercept
rr = r_value**2
print(rr)

0.957715666948


In [57]:
foorm = str(form) +' '+ 'R² ='+' '+ str(rr)
lege = textwrap.fill(foorm,len(str(form)))
palavra = 'Comparação dos valores de Temperatura CTD MB x CTD FURG' +' '+est 
legen = textwrap.fill(palavra,40)
print(legen)

plt.figure(figsize=(9, 10))
plt.scatter(X,Y, color='r', s=50, label='Dados Observados')
#plt.plot(X,Y,'og', s=4, label='Dados Observados')
plt.plot([0,50],[b,m*50+b],'-b',linewidth=4, label=lege)
plt.title(legen, fontsize = 20)
plt.xlabel('CTD Observado', fontsize = 15)
plt.ylabel('XBT Interpolado', fontsize = 15)
plt.xlim(minixbt - 1,maxixbt + 1)
plt.ylim(minictd - 1,maxictd + 1)
plt.legend(loc='upper left', shadow=True, fontsize='large')
plt.grid()
plt.show()

Comparação dos valores de Temperatura
CTD MB x CTD FURG MCT030


In [58]:
dados = pd.DataFrame(data = X, columns= ['CTD_observado'])
dados['XBT_interpolado'] = Y 
dados.to_excel(r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\Analises\CTDxCTD\perfil_'+ num +r'\perfil_sal_REGRE'+ num +'.xlsx',index=True,header=True)



# Regressao e Perfis

In [29]:
palavras = 'Perfil CTD MB x CTD FURG Estação'+' '+est
legens = textwrap.fill(palavras,25)
print(legens)

Perfil CTD MB x CTD FURG
Estação MCT007


In [62]:
palavras = 'Perfil CTD MB x CTD FURG Estação'+' '+est
legens = textwrap.fill(palavras,25)
print(legens)
foorm = str(form) +' '+ 'R² ='+' '+ str(rr)
lege = textwrap.fill(foorm,len(str(form)))
palavra = 'Comparação dos valores de Salinidade CTD MB x CTD FURG' +' '+est 
legen = textwrap.fill(palavra,38)
print(legen)


plt.figure(figsize=(6, 8))
plt.plot(df['temp_xbt'],df['prof_xbt'], 'b', label="Perfil CTD FURG", linewidth=5)
plt.plot(df2['temp_ctd'], df2['prof_ctd'], 'r', label="Perfil CTD MB", linewidth=5)
plt.ylim(100, 0)
plt.xlim(30, 40)
plt.title(legens, fontsize = 20)
plt.xlabel('Salinidade[PSU]', fontsize = 15)
plt.ylabel('Profundidade[m]', fontsize = 15)
plt.legend(loc='lower left', shadow=True, fontsize='large')
plt.grid()
#plt.show()

plt.figure(figsize=(9, 10))
plt.scatter(X,Y, color='r', s=50, label='Dados Observados')
#plt.plot(X,Y,'og', s=4, label='Dados Observados')
plt.plot([0,50],[b,m*50+b],'-b',linewidth=4, label=lege)
plt.title(legen, fontsize = 20)
plt.xlabel('CTD MB', fontsize = 15)
plt.ylabel('CTD FURG', fontsize = 15)
plt.xlim(minixbt - 1,maxixbt + 1)
plt.ylim(minictd - 1,maxictd + 1)
plt.legend(loc='upper left', shadow=True, fontsize='large')
plt.grid()
plt.show()

Perfil CTD MB x CTD FURG
Estação MCT030
Comparação dos valores de Salinidade
CTD MB x CTD FURG Total


# REGRESSAO TOTAL

In [61]:
#####regressao Total

regtotal = pd.read_excel(r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\Analises\CTDxCTD\perfil_sal_REGRE_TOTAL.xlsx')
regtotal.head(4)

,CTD_observado,XBT_interpolado
0,34.1945,33.8420
1,34.1801,33.8370
2,34.1803,33.8324
3,34.1805,33.8319


In [63]:
minixbt = regtotal['XBT_interpolado'].min()
minictd = regtotal['CTD_observado'].min()
maxixbt = regtotal['XBT_interpolado'].max()
maxictd = regtotal['CTD_observado'].max()

npMatrix = np.matrix(regtotal)
XX, YY = npMatrix[:,1], npMatrix[:,0]
mdl = LinearRegression().fit(X,Y) # either this or the next line
#mdl = LinearRegression().fit(filtered_data[['x']],filtered_data.y)
m = mdl.coef_[0]
b = mdl.intercept_
#print("formula: y = {0}x + {1}".format(m, b)) # following slope intercept form 
stm = str(m)
stb = str(b)
form = ("y = {0}x + {1}".format(stm[1:len(stm)-1], stb[1:len(stb)-1]))
print(form)

x = regtotal['CTD_observado']
y = regtotal['XBT_interpolado']
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
a = slope
bb = intercept
rr = r_value**2
print(rr)

palavra = 'Comparação dos valores de Salinidade CTD MB x CTD FURG Total'# +' '+est 
legen = textwrap.fill(palavra,38)
print(legen)

plt.figure(figsize=(9, 10))
plt.scatter(XX,YY, color='r', s=50, label='Dados Observados')
#plt.plot(X,Y,'og', s=4, label='Dados Observados')
plt.plot([0,50],[b,m*50+b],'-b',linewidth=4, label=lege)
plt.title(legen, fontsize = 20)
plt.xlabel('CTD MB', fontsize = 15)
plt.ylabel('CTD FURG', fontsize = 15)
plt.xlim(minixbt - 1,maxixbt + 1)
plt.ylim(minictd - 1,maxictd + 1)
plt.legend(loc='upper left', shadow=True, fontsize='large')
plt.grid()
plt.show()

y =  0.85579431x +  4.73141176
0.960373741109
Comparação dos valores de Salinidade
CTD MB x CTD FURG Total
